In [1]:
from MEC import Circle, Point, welzl
import numpy as np
import math

def calculate_minimum_enclosing_circle(points):
    """
    Calculate the minimum enclosing circle for a set of points using Welzl's algorithm.
    Returns the center and radius of the circle.
    """
    mec = welzl(points)  
    center=[]
    center.append(mec.C.X)
    center.append(mec.C.Y)
    return center, mec.R

def _rotation_matrix(theta):
    """Creates a 2D rotation matrix for a given angle in radians."""
    return np.array([
        [math.cos(theta), -math.sin(theta)],
        [math.sin(theta), math.cos(theta)]
    ])

def from_rotated_box(xc, yc, w, h, theta, frame_size=None):
    """
    Constructs a rotated bounding box from its center, dimensions, and rotation.

    Args:
        xc (float): x-center coordinate
        yc (float): y-center coordinate
        w (float): box width
        h (float): box height
        theta (float): rotation angle in radians
        frame_size (tuple or None): (width, height) of the frame. If provided, points are normalized.

    Returns:
        list of tuples: Normalized (x, y) coordinates of the rotated bounding box corners
    """
    # Define the rotation matrix
    R = _rotation_matrix(theta)
    
    # Define the local coordinates of the bounding box corners
    local_points = np.array([
        [ w / 2,  h / 2],
        [-w / 2,  h / 2],
        [-w / 2, -h / 2],
        [ w / 2, -h / 2]
    ])  # shape (4, 2)
    
    # Apply rotation
    rotated_points = R.dot(local_points.T).T  # shape (4, 2)
    
    # Apply translation to the center
    rotated_translated_points = rotated_points + np.array([xc, yc])  # shape (4,2)
    
    # If frame_size is provided, normalize the points
    if frame_size is not None:
        rotated_translated_points = rotated_translated_points / np.array(frame_size)
    
    # Convert to list of tuples
    normalized_points = rotated_translated_points.tolist()
    
    return normalized_points

# good

In [20]:
from utils import calculate_euclidean_distance, calculate_real_width
from skeletonization import skeletonize_mask,create_filled_binary_mask, skeletonize_mask, find_longest_path
import cv2
import fiftyone as fo
import numpy as np

def process_segmentations(segmentation_path):
    """
    Process the segmentations from the TXT file, calculate the minimum enclosing circle for each prawn.
    """
    segmentations = []
    skeletons=[]
    hulls=[]
    skeletons_straight=[]
    skeletons_straight_2=[]
    seg_closeds=[]
    skeletons_2=[]
    box_diagonal=[] 
    boxes=[]
    masks=[]






    
    # Open the segmentation file and process each line
    with open(segmentation_path, 'r') as file:
        for line in file:
            coords = [float(x) for x in line.strip().split()]
            coords_array = np.array(coords).reshape(-1, 2)
            

            
            binary_mask_no = create_filled_binary_mask(coords_array, 360, 640, gaussian_blur=False) 
            
            if np.sum(binary_mask_no) == 0:   
                continue





            binary_dilated = cv2.dilate(binary_mask_no, np.ones((15, 15), np.uint8), iterations=1)     


            #contour dilated
            contures_dil, _ = cv2.findContours(binary_dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    



            prawn_conture_dil = max(contures_dil, key=cv2.contourArea)

            coords_contour_dil = np.column_stack(prawn_conture_dil).flatten()

            normalized_coords_bin=[(coords_contour_dil[i]/640, coords_contour_dil[i+1]/360) for i in range(0, len(coords_contour_dil), 2)]  # Extract points (x, y)


            masks.append(fo.Polyline(
                points=[normalized_coords_bin],
                closed=True,
                filled=False,
            ))


            
            thinned_2=skeletonize_mask(binary_mask_no)

            # skeleton = skeletonize_mask(binary_mask)
            skeleton_2 = thinned_2
            skeleton_coords_2 = np.column_stack(np.nonzero(skeleton_2))
            normalized_coords_2,max_length_2 = find_longest_path(skeleton_coords_2,(360,640),(2988,5312))

            normalized_coords_2 = [(x, y) for y, x in normalized_coords_2]  # Convert to (y, x) format

            #only the first and last points of the skeleton
            normalized_coords_straight_2 = [normalized_coords_2[0], normalized_coords_2[-1]]  





             
            #convex hull diameter
            contures, _ = cv2.findContours(binary_mask_no, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            
            prawn_conture = max(contures, key=cv2.contourArea)  

            # Compute the minimum area rectangle enclosing the shrimp
            rect = cv2.minAreaRect(prawn_conture)
            box_points = cv2.boxPoints(rect)
            box_points = np.int0(box_points)

            original_size = (640, 360)
            new_size = (5312, 2988)

            # Scaling factors
            scale_x = new_size[0] / original_size[0]  # 5312 / 640
            scale_y = new_size[1] / original_size[1]  # 2988 / 640

            box_points_scaled = np.array([(point[0] * scale_x, point[1] * scale_y) for point in box_points])

            width= calculate_euclidean_distance(box_points_scaled[0], box_points_scaled[1])
            height= calculate_euclidean_distance(box_points_scaled[1], box_points_scaled[2])

            max_length_box=max(width,height)


        
            # Convert theta from degrees to radians for FiftyOne
            theta_radians = np.deg2rad(rect[2])
            # normalized_bounding_box = [(box_points[i][0]/640, box_points[i][1]/640) for i in range(0, len(box_points))] 
            
            # image_center_x = 640 / 2

            # xc_adjusted = rect[0][0] - image_center_x

            # Extract points (x, y) 
            box=fo.Polyline.from_rotated_box(
                xc=rect[0][0] ,
                yc=rect[0][1],
                w=rect[1][0],
                h=rect[1][1],
                theta =theta_radians,
                frame_size=(640, 360)

            )


            boxes.append(box)


            hull_points = cv2.convexHull(prawn_conture, returnPoints=True)


            
# Scaling factors to convert from 640x640 to 5312x2988
            scale_x = 5312 / 640
            scale_y = 2988 / 360

        # Scale the points to the new resolution
            scaled_hull_points = []
            for point in hull_points:
                x, y = point[0]
                scaled_x = x * scale_x
                scaled_y = y * scale_y
                scaled_hull_points.append([scaled_x, scaled_y])

            # Convert to numpy array for easier handling
            scaled_hull_points = np.array(scaled_hull_points, dtype=np.float32)

            # Now, find the maximum Euclidean distance (convex hull diameter) using the scaled points
            max_distance = 0
            point1 = None
            point2 = None

            # Loop through all pairs of scaled points to find the maximum distance
            for i in range(len(scaled_hull_points)):
                for j in range(i + 1, len(scaled_hull_points)):
                    distance = calculate_euclidean_distance(scaled_hull_points[i], scaled_hull_points[j])
                    if distance > max_distance:
                        max_distance = distance
                        point1 = scaled_hull_points[i]
                        point2 = scaled_hull_points[j]

            # The result is max_distance (in pixels) in the 5312x2988 image


            normalzied_points_hull = [(point1[0]/5312, point1[1]/2988), (point2[0]/5312, point2[1]/2988)]  # Extract points (x, y)

            hull=fo.Polyline(
                points=[normalzied_points_hull],
                closed=False,
                filled=False,
                max_length=max_distance
            )

        # skeleton_straight=fo.Polyline(
        #     points=[normalized_coords_straight],
        #     closed=False,
        #     filled=False,
        #     max_length=max_length
        # )
        # skeletons_straight.append(skeleton_straight)

            skeleton_straight_2=fo.Polyline(
                points=[normalized_coords_straight_2],
                closed=False,
                filled=False,
                max_length=max_length_2,
                
            )
            skeletons_straight_2.append(skeleton_straight_2)




            hulls.append(hull)

            # skeleton=fo.Polyline(
            #     points=[normalized_coords],
            #     closed=False,
            #     filled=False,
            #     max_length=max_length
            # )

            # skeletons.append(skeleton)
            
            skeleton_2=fo.Polyline( 
                points=[normalized_coords_2],
                closed=False,
                filled=False,
                max_length=max_length_2)
            skeletons_2.append(skeleton_2)
              # Convert the line to a list of floats
            normalzied_points = [(coords[i]/640, coords[i + 1]/360) for i in range(0, len(coords), 2)]  # Extract points (x, y)
            # points = [Point(x*5312, y*2988) for x, y in normalzied_points] 
            

            scaled_contour = []
            for point in prawn_conture:
                scaled_x = int(point[0][0] * scale_x)
                scaled_y = int(point[0][1] * scale_y)
                scaled_contour.append([[scaled_x, scaled_y]])

            scaled_contour = np.array(scaled_contour, dtype=np.int32)

             # Convert to Point objects    
            # Calculate the minimum enclosing circle (center and radius)
            center, radius = cv2.minEnclosingCircle(scaled_contour)



            diameter = radius * 2


            #center in 640x360
            center_640 = (center[0] / scale_x, center[1] / scale_y)
        

            segmentation = fo.Polyline(
                points=[normalzied_points],
                closed=True,
                filled=False,
                diameter=diameter,
                center=center_640,
                max_length_skeleton=max_length_2,
                max_length_hull=max_distance,
                max_length_box=max_length_box
            )

            #smooth segmentation  wirh closing
            # seg_closed=fo.Polyline(
            #     points=[normalized_coords_bin],
            #     closed=True,
            #     filled=False,
            #     max_length=max_length
            # )

            # seg_closeds.append(seg_closed)                


            segmentations.append(segmentation)







                     # Store the segmentation information (center, radius, and diameter)

    return segmentations,skeletons,hulls, skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks

In [21]:
import fiftyone.core.labels as fol
from tqdm import tqdm
import fiftyone as fo
import os

def process_images(image_paths, prediction_folder_path, dataset,tag):
    print("Processing images...")
    
    """
    Processes images by matching segmentation with bounding boxes and calculating prawn sizes.
    """
    for image_path in tqdm(image_paths):
        # filename = os.path.splitext(os.path.basename(image_path))[0]
        

           
        # prediction_txt_path = os.path.join(prediction_folder_path, f"{os.path.basename(image_path).split('.')[0]}_segmentations.txt")

        core_name = os.path.splitext(os.path.basename(image_path))[0]

        # Construct the path to the corresponding segmentation file



        #check if txt exist

        if not os.path.exists(os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")):
            print(f"No segmentation file found for {core_name}")
            continue



        prediction_txt_path = os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")

        # core_name=filename.split('.')[0]
        # # Construct the path to the prediction (segmentation) file
        # prediction_txt_path = os.path.join(prediction_folder_path, f"{core_name}_segmentations.txt")
        # if not os.path.exists(prediction_txt_path):
        #     print(f"No segmentation file found for {filename}")
        #     continue


        # Parse the segmentations to get the minimum enclosing circles
        segmentations,skeletons,hulls,skeletons_straight,seg_closeds,skeletons_2,skeletons_straight_2,boxes,masks = process_segmentations(prediction_txt_path)

        # Save the modified image (with circles drawn)

        # Create a new sample for FiftyOne
        sample = fo.Sample(filepath=image_path)

        # Iterate over each bounding box in the filtered data
        sample["segmentations"] = fol.Polylines(polylines=segmentations)

        # sample["skeletons"] = fol.Polylines(polylines=skeletons)

        sample["hulls"] = fol.Polylines(polylines=hulls)    

        # sample["skeletons_straight"] = fol.Polylines(polylines=skeletons_straight)

        # sample['seg_closeds']=fol.Polylines(polylines=seg_closeds)
        
        sample['skeletons_no_smooth']=fol.Polylines(polylines=skeletons_2)

        sample["skeletons_straight_no_smooth"] = fol.Polylines(polylines=skeletons_straight_2)

        sample['boxes']=fol.Polylines(polylines=boxes)

        sample['masks']=fol.Polylines(polylines=masks)
        # Add the processed sample to the FiftyOne dataset
        sample.tags.append(tag)
        dataset.add_sample(sample)

In [27]:
import math
def process_detection_by_circle(segmentation, sample,i,dataframe):
    """
    Process the prawn detection based on the enclosing circle's diameter.
    Update the filtered dataframe with the real-world size of the prawn.
    """
    poly=segmentation

    if 'right' in str(sample.tags[0]):
        height_mm =700-30
        focal_length = 23.64    
    else:
        height_mm=410-30
        focal_length=24.72

    fov=75
    predicted_hull_length=poly['max_length_hull']
    
    FOV_width = 2 * height_mm * math.tan(math.radians(fov / 2))
    hull_length_fov = FOV_width * predicted_hull_length / 5312

    if hull_length_fov < 120 or hull_length_fov > 220:

        return poly

    

    if hull_length_fov < 175:
        best_true_length = 143
        if hull_length_fov < 143 and 'smaller than 143mm' not in sample.tags:
            sample.tags.append('smaller than 143mm')
            dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'true_length'] = '<143mm'


    else:
        best_true_length = 180
        if hull_length_fov < 180 and 'smaller than 180mm' not in sample.tags:
            sample.tags.append('smaller than 180mm')
            dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'true_length'] = '<180mm'

    # for true_length in [143, 180]:
        
    

       
    #     error_percentage_hull_fov = abs(hull_length_fov - true_length) / true_length * 100
        


    #     if error_percentage_hull_fov < min_error_percentage:
    #         min_error_percentage = error_percentage_hull_fov
    #         best_true_length = true_length

    if best_true_length == 143:
        height_mm = height_mm
    else:
        height_mm = height_mm - 15


  
    # print(f"height_mm: {height_mm}, tag: {tag}")

    # focal_length = 24.22  # Camera focal length
    pixel_size = 0.00716844  # Pixel size in mm

    

    fov=75
    FOV_width=2*height_mm*math.tan(math.radians(fov/2))


    # Get the diameter of the circle in pixels
    predicted_diameter_pixels = poly['diameter']


    predicted_skeleton_length=poly['max_length_skeleton']  

    predicted_hull_length=poly['max_length_hull']

     
    predicted_box_length=poly['max_length_box']

    center=poly['center']




    #add true lenght to datafram
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'true_length'] = best_true_length


    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'center'] = str(center)

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Hull_Length_pixels'] = predicted_hull_length
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Diameter_pixels'] = predicted_diameter_pixels
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Skeleton_Length_pixels'] = predicted_skeleton_length
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Box_Length_pixels'] = predicted_box_length



    # Calculate the real-world prawn size using the box
    focal_box = calculate_real_width(focal_length, height_mm, predicted_box_length, pixel_size) 


    focal_hull = calculate_real_width(focal_length, height_mm, predicted_hull_length, pixel_size)    
    # Calculate the real-world prawn size using the enclosing circle's diameter
    focal_MEC = calculate_real_width(focal_length, height_mm, predicted_diameter_pixels, pixel_size)

    focal_ske = calculate_real_width(focal_length, height_mm, predicted_skeleton_length, pixel_size)    


    # Update the filtered dataframe with the real-world size of the prawn
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Focal_Box'] = focal_box
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Focal_Hull'] = focal_hull
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Focal_MEC'] = focal_MEC
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Focal_Skeleton'] = focal_ske



    hull_length_fov=FOV_width*predicted_hull_length/5312
    diameter_length_fov=FOV_width*predicted_diameter_pixels/5312
    skeleton_length_fov=FOV_width*predicted_skeleton_length/5312

    box_length_fov=FOV_width*predicted_box_length/5312

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Hull_Length_fov'] = hull_length_fov
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Diameter_fov'] = diameter_length_fov
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Skeleton_Length_fov'] = skeleton_length_fov
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Box_Length_fov'] = box_length_fov



    error_percentage_hull_fov = abs(hull_length_fov - best_true_length) / best_true_length * 100

    error_percentage_skeleton_fov = abs(skeleton_length_fov - best_true_length) / best_true_length * 100  


    error_percentage_box_fov = abs(box_length_fov - best_true_length) / best_true_length * 100


    error_percentage_MEC_fov = abs(diameter_length_fov - best_true_length) / best_true_length * 100

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_Hull_fov'] = error_percentage_hull_fov
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_Skeleton_fov'] = error_percentage_skeleton_fov
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_Box_fov'] = error_percentage_box_fov

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_MEC_fov'] = error_percentage_MEC_fov

    error_percentage_hull_focal = abs(focal_hull - best_true_length) / best_true_length * 100

    error_percentage_skeleton_focal = abs(focal_ske - best_true_length) / best_true_length * 100

    error_percentage_box_focal = abs(focal_box - best_true_length) / best_true_length * 100

    error_percentage_MEC_focal = abs(focal_MEC - best_true_length) / best_true_length * 100

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_Hull_focal'] = error_percentage_hull_focal
    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_Skeleton_focal'] = error_percentage_skeleton_focal

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_Box_focal'] = error_percentage_box_focal

    dataframe.loc[(dataframe['index'] == i) & (dataframe['image_path'] == sample.filepath), 'Error_percentage_MEC_focal'] = error_percentage_MEC_focal  





    closest_detection_label = f'index:{i} ,true length: {best_true_length:.2f}mm, MPError_hull: {error_percentage_hull_fov:.2f}%, pred length: {hull_length_fov:.2f}mm ,error percentage MEC{error_percentage_MEC_fov:.2f}% ,pred MEC {diameter_length_fov:.2f} ,error percentage skeleton: {error_percentage_skeleton_fov:.2f}%, , pred length: {skeleton_length_fov:.2f}cm, error percentage box: {error_percentage_box_fov:.2f}%, pred length: {box_length_fov:.2f}mm, '
    # Update the filtered dataframe with the real-world size of the prawn



    if error_percentage_hull_fov > 25 and f'>25% error' not in sample.tags:

        sample.tags.append(f'>25% error')

    elif error_percentage_hull_fov > 15 and f'15-25% error' not in sample.tags:

        sample.tags.append(f'15-25% error')

    elif error_percentage_hull_fov > 10  and f'10-15% error' not in sample.tags:

        sample.tags.append(f'10-15% error')

    elif error_percentage_hull_fov < 10 and f'5-10% error' not in sample.tags:

        sample.tags.append(f'5-10% error')
    
    elif error_percentage_hull_fov < 5 and f'<5% error' not in sample.tags:

            
        sample.tags.append(f'<5% error')





    poly.label = closest_detection_label 
    return poly
  

In [28]:
molt_right_image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\good\640360\right\color_images"

molt_square_image_path=r"c:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\good\640360\square\color_images"

molt_prediction_right=r'C:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\good\640360\right\segmentations'

molt_prediction_sqaure=r'c:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\good\640360\square\segmentations'


import pandas as pd


import fiftyone as fo
dataset = fo.Dataset("molt", overwrite=True)

# Load the dataset
right_image_paths = [os.path.join(molt_right_image_path, image) for image in os.listdir(molt_right_image_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

square_image_paths = [os.path.join(molt_square_image_path, image) for image in os.listdir(molt_square_image_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

for molt_prediction in [molt_prediction_right,molt_prediction_sqaure]:
    prediction_paths_text = [os.path.join(molt_prediction, txt) for txt in os.listdir(molt_prediction) if txt.endswith('.txt')]




    tag=molt_prediction.split('\\')[-2]
    
  
    if 'right' in molt_prediction:
        image_paths=right_image_paths
        process_images(image_paths, molt_prediction, dataset,tag)
    else:
        image_paths=square_image_paths
        process_images(image_paths, molt_prediction, dataset,tag)
    # Process segmentations

dataframe=pd.DataFrame(columns= ['image_path','index','pond'])

for sample in dataset:
    # Access the polylines for each sample
    for i, segmentation in enumerate(sample["segmentations"].polylines):
        
        dataframe=dataframe._append({'image_path':sample.filepath,'index':i,'pond':tag},ignore_index=True)
        # Process and modify the segmentation
        updated_segmentation = process_detection_by_circle(segmentation,sample,i,dataframe)

        # Save the updated segmentation back into the sample
        sample["segmentations"].polylines[i] = updated_segmentation

        sample.save()


dataframe.to_csv('molt.csv')
session = fo.launch_app(dataset,port=5152)

Processing images...


  0%|          | 0/66 [00:00<?, ?it/s]C:\Users\gbo10\AppData\Local\Temp\ipykernel_6608\3297858279.py:94: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)
100%|██████████| 66/66 [00:07<00:00,  9.40it/s]


Processing images...


100%|██████████| 56/56 [00:05<00:00,  9.78it/s]



Could not connect session, trying again in 10 seconds

Could not connect session, trying again in 10 seconds



Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



KeyboardInterrupt: 

In [33]:
#print workspace

print (session.view)

Dataset:     molt
View name:   molt
Media type:  image
Num samples: 122
Sample fields:
    id:                           fiftyone.core.fields.ObjectIdField
    filepath:                     fiftyone.core.fields.StringField
    tags:                         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    segmentations:                fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polylines)
    hulls:                        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polylines)
    skeletons_no_smooth:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polylines)
    skeletons_straight_no_smooth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polylines)
    boxes:                        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Polylines)
    masks:        

In [34]:


# Export the view to CSV
export_path = "saved_view_data.csv"
session.view.export(
    export_path,
    dataset_type=fo.types.CSVDataset,
    fields=["id", "filepath", "tags", "metadata"]  # Adjust fields as needed
)


 100% |█████████████████| 122/122 [510.6ms elapsed, 0s remaining, 238.9 samples/s]      


In [6]:
from tqdm import tqdm

image_paths=r'C:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\unditorted'

output_path=r'C:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\unditorted\undistorted_resized'

image_paths = [os.path.join(image_paths, image) for image in os.listdir(image_paths) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

for image_path in tqdm(image_paths):
    #resize the image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (640, 640))
    cv2.imwrite(os.path.join(output_path, os.path.basename(image_path)), img)

100%|██████████| 20/20 [00:06<00:00,  2.92it/s]


In [7]:
import cv2
import numpy as np
from tqdm import tqdm
import os


molt_image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\all molt\undistorted"


output_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\all molt\undistorted\first_process"


os.makedirs(output_path, exist_ok=True)

# do the code below on entire folder
image_paths = [os.path.join(molt_image_path, image) for image in os.listdir(molt_image_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]

for image_path in tqdm(image_paths):

# Load the image
    image = cv2.imread(image_path)

    # Step 1: Convert to grayscale
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    white_mask=cv2.inRange(image, (60, 60, 60), (255, 255, 255))

    # cv2.imwrite("white_mask.png", white_mask)    



    very_white_mask = cv2.inRange(image, (150, 150, 150), (255, 255, 255))


    image[very_white_mask == 255] = [0, 0, 0]  # Change white to gray


    # cv2.imwrite("very_white_mask.png", image)


    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    # _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # Save the result
    # cv2.imwrite("binary_image.png", binary)


    # cv2.imwrite("gray.png", gray)


    # white_mask_from_binary = cv2.bitwise_not(binary)  # Invert the binary mask to get white areas

    # cv2.imwrite("white_mask_from_binary.png", white_mask_from_binary)


    gray_color = np.array([52, 66, 79])  # RGB value for gray
    # image[white_mask == 255] = gray_color

    # Step 2: Threshold the image to isolate black areas (under prawns)
    # Adjust threshold values depending on how black the segments are
    _, black_mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)  # Isolating black areas (threshold of 50)


    # cv2.imwrite("black_mask.png", black_mask)

    # Step 3: Create a bluish image to apply to black areas
    bluish_image = image.copy()
    bluish_image[:] = [212, 156, 31]  # BGR for blue


    #white mask on the black segments
    # white_on_black = cv2.bitwise_and(white_mask_from_binary, black_mask)  # Isolate white areas on the black segments

    # cv2.imwrite("white_on_black.png", white_on_black)


    # # Step 6: Convert white to gray in those areas
    # # gray_color = np.array([128, 128, 128])  # RGB value for gray
    # image[white_on_black==0] = gray_color  # Change white on black to gray


    image[white_mask == 255] = gray_color

    # Step 4: Apply bluish color to the black regions
    image[black_mask == 255] = bluish_image[black_mask == 255]





    # Step 5: Isolate white areas (likely prawns) within the black segments
    # Detect white pixels in the original image (on the black segments only)
    # white_on_black = cv2.bitwise_and(white_mask, black_mask)  # Isolate white areas on the black segments

    # Step 6: Convert white to gray in those areas
    # gray_color = np.array([128, 128, 128])  # RGB value for gray
    # image[white_on_black == 255] = gray_color  # Change white on black to gray

    # Step 7: Display the final image
    # cv2.imshow("Result", image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # Save the result


    #resize the image
    resized_image = cv2.resize(image, (640, 360))



    cv2.imwrite(os.path.join(output_path, os.path.basename(image_path)), resized_image)


100%|██████████| 191/191 [03:31<00:00,  1.11s/it]


In [8]:



image = cv2.imread(r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\image processed\vlcsnap-2024-09-21-02h23m31s263.png")

    # Step 1: Convert to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


white_mask=cv2.inRange(image, (60, 60, 60), (255, 255, 255))

# cv2.imwrite("white_mask.png", white_mask)    



very_white_mask = cv2.inRange(image, (150, 150, 150), (255, 255, 255))


image[very_white_mask == 255] = [0, 0, 0]  # Change white to gray


# cv2.imwrite("very_white_mask.png", image)


gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


# _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Save the result
# cv2.imwrite("binary_image.png", binary)


# cv2.imwrite("gray.png", gray)


# white_mask_from_binary = cv2.bitwise_not(binary)  # Invert the binary mask to get white areas

# cv2.imwrite("white_mask_from_binary.png", white_mask_from_binary)


gray_color = np.array([132, 129, 122])  # RGB value for gray
# image[white_mask == 255] = gray_color

# Step 2: Threshold the image to isolate black areas (under prawns)
# Adjust threshold values depending on how black the segments are
_, black_mask = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)  # Isolating black areas (threshold of 50)


# cv2.imwrite("black_mask.png", black_mask)

# Step 3: Create a bluish image to apply to black areas
bluish_image = image.copy()
bluish_image[:] = [212, 156, 31]  # BGR for blue


#white mask on the black segments
# white_on_black = cv2.bitwise_and(white_mask_from_binary, black_mask)  # Isolate white areas on the black segments

# cv2.imwrite("white_on_black.png", white_on_black)


# # Step 6: Convert white to gray in those areas
# # gray_color = np.array([128, 128, 128])  # RGB value for gray
# image[white_on_black==0] = gray_color  # Change white on black to gray


image[white_mask == 255] = gray_color

# Step 4: Apply bluish color to the black regions
image[black_mask == 255] = bluish_image[black_mask == 255]





# Step 5: Isolate white areas (likely prawns) within the black segments
# Detect white pixels in the original image (on the black segments only)
# white_on_black = cv2.bitwise_and(white_mask, black_mask)  # Isolate white areas on the black segments

# Step 6: Convert white to gray in those areas
# gray_color = np.array([128, 128, 128])  # RGB value for gray
# image[white_on_black == 255] = gray_color  # Change white on black to gray

# Step 7: Display the final image
# cv2.imshow("Result", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Save the result

#resize the image
cv2.imwrite(r"vlcsnap-2024-09-21-02h23m31s263.png", image)

True

In [9]:
import cv2
import numpy as np

# Load a binary image

image = cv2.imread(r'C:\Users\gbo10\Videos\research\counting_research_algorithms\fifty_one\measurements\vlcsnap-2024-09-21-02h23m31s263.png')

# # Convert to grayscale
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# #save the gray image
# cv2.imwrite("gray_image.png", gray)
gray_color = np.array([122, 129, 132])  # RGB value for gray

lower_bound = gray_color -100 # Allow a small tolerance
upper_bound = gray_color + 100

# Create a mask where the pixels are within the range of the gray color
mask = cv2.inRange(image, lower_bound, upper_bound)

# Invert the mask to get the pixels that are NOT the gray color

cv2.imwrite("mask.png", mask)

# # Make the non-gray pixels black by applying the inverted mask
# result = image.copy()
# # result[mask_inv == 255] = [0, 0, 0] 


#thershold the image using gray_color = np.array([132, 129, 122])  # RGB value for gray


# cv2.imwrite("result.png", result)

contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(f"Found {len(contours)} contours")

conutres1=[]

i=0
# Iterate through contours
for contour in contours:
    # Calculate area and perimeter (arc length)
    area = cv2.contourArea(contour)
    # perimeter = cv2.arcLength(contour, True)
   
    if area > 90000:
        

        conutres1.append(contour)

        print(f"Contour {i}: Area: {area:.2f}")
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
        i+=1


#     #Draw the contour

#what is not in the contures1 paint in azure   
    
   
# # Save the output image
cv2.imwrite('ellipses_output.png', image)




# Find contours
# contours, hierarchy = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# # Draw contours on the original image
# output_image = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)  # Convert grayscale to BGR for color drawing
# cv2.drawContours(output_image, contours, -1, (0, 255, 0), 2)  # Draw all contours in green

# Save the output image
# cv2.imwrite('contours_output.png', output_image)


Found 1824 contours
Contour 0: Area: 579477.00
Contour 1: Area: 544803.50


True

In [10]:

import cv2
import numpy as np

# Load the image
image = cv2.imread(r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\all molt\undistorted\first_process\right\azure\undistorted_GX010191_37_392.jpg")

# Define the color to paint the background (Azure color in BGR format)
azure_color = [212, 156, 31]  # Azure in BGR (Blue, Green, Red)

# Define gray color and tolerance
gray_color = np.array([52, 66, 79])  # RGB value for gray
lower_bound = gray_color - 50
upper_bound = gray_color + 50


# Create a mask for the gray color
mask = cv2.inRange(image, lower_bound, upper_bound)

# Find contours in the mask
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours based on area and create a new mask for the contours
contours_mask = np.zeros_like(mask)  # A black mask of the same size as the original image

# Iterate through contours and draw only the ones with sufficient area
for contour in contours:
    area = cv2.contourArea(contour)
    if area > 500: 
        print (area)
        # Draw the contours on the black mask
        cv2.drawContours(contours_mask, [contour], -1, 255, thickness=cv2.FILLED)  # Fill the contours in white

# Invert the contours mask (make contours black and the rest white)
contours_mask_inv = cv2.bitwise_not(contours_mask)

# Create a background image filled with the azure color
azure_background = np.full_like(image, azure_color, dtype=np.uint8)

# Mask out the contour areas and paint the rest azure
result = cv2.bitwise_and(image, image, mask=contours_mask)  # Keep the original image inside the contours
azure_areas = cv2.bitwise_and(azure_background, azure_background, mask=contours_mask_inv)  # Paint everything else azure

# Combine the original image (inside contours) with the azure background (outside contours)
final_result = cv2.add(result, azure_areas)

# Save and display the final result
cv2.imwrite("result_azure_background.png", final_result)



589.5
32496.5
12145.0
3556.5
1674.5
638.0


True

In [11]:
    
import cv2
import numpy as np
import os
from tqdm import tqdm

molt_image_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\all molt\undistorted\first_process\square"


output_path = r"C:\Users\gbo10\OneDrive\measurement_paper_images\molt\all molt\undistorted\first_process\square\azure"

os.makedirs(output_path, exist_ok=True) 

image_paths = [os.path.join(molt_image_path, image) for image in os.listdir(molt_image_path) if image.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff'))]


for image_path in tqdm(image_paths):

    # Load the image
    image = cv2.imread(image_path)
    azure_color = [212, 156, 31]  # Azure in BGR (Blue, Green, Red)

    # Define gray color and tolerance
    gray_color = np.array([52, 66, 79])  # RGB value for gray
    lower_bound = gray_color - 50
    upper_bound = gray_color + 50

    # Create a mask for the gray color
    mask = cv2.inRange(image, lower_bound, upper_bound)

    cv2.imwrite("mask.png", mask)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area and create a new mask for the contours
    contours_mask = np.zeros_like(mask)  # A black mask of the same size as the original image

    # Iterate through contours and draw only the ones with sufficient area
    for contour in contours:
        area = cv2.contourArea(contour)
        if area >800:  
            
     
            # Filter by area
            # Draw the contours on the black mask
            cv2.drawContours(contours_mask, [contour], -1, 255, thickness=cv2.FILLED)  # Fill the contours in white

    # Invert the contours mask (make contours black and the rest white)
    contours_mask_inv = cv2.bitwise_not(contours_mask)

    # Create a background image filled with the azure color
    azure_background = np.full_like(image, azure_color, dtype=np.uint8)

    # Mask out the contour areas and paint the rest azure
    result = cv2.bitwise_and(image, image, mask=contours_mask)  # Keep the original image inside the contours
    azure_areas = cv2.bitwise_and(azure_background, azure_background, mask=contours_mask_inv)  # Paint everything else azure

    # Combine the original image (inside contours) with the azure background (outside contours)
    final_result = cv2.add(result, azure_areas)

    #Save the result
    cv2.imwrite(os.path.join(output_path, os.path.basename(image_path)), final_result)



  0%|          | 0/93 [00:00<?, ?it/s]

100%|██████████| 93/93 [00:02<00:00, 41.29it/s]
